In [4]:
import pandas as pd

In [5]:
df=pd.read_csv("salary_indeed.csv")

In [6]:
df["Salary"][0]

nan

In [7]:
df['Title'].unique().tolist()

['Développeuse / Développeur Stack Elastic Search',
 'Développeur Front End F/H',
 'Data Analyst (H/F) - Alternance',
 "INTERNSHIP POSITION, Evaluation of the aircraft data traffic's cost (m/f)",
 'Des Développeurs Java JEE (H/F)',
 'LEAD DEVELOPPEUR PYTHON F/H',
 'Consultant Confirmé BI / Data Intelligence (H/F)',
 'Consultant Data (h/f)',
 'INGÉNIEUR DÉVELOPPEUR C++ / JAVA F/H',
 'Devops – Spécialiste Openstack H/F',
 'Développeur Web et Bases de Données F/H',
 'Data Engineer - Java 8 - H/F',
 'DEVOPS junior Data Management Middleware F/H',
 'INGENIEUR DEVELOPPEUR FIRMWARE',
 'Data Engineer - Santé',
 'Devops F/H',
 'Développeur Angular & C# H/F',
 'Data Scientist H/F',
 'Cloud Data Architect Manager – Secteur de la santé (H/F)',
 'BUSINESS DEVELOPER BTOB - DATA INTELLIGENCE (H/F)',
 'DATA MANAGER EXPERIMENTE',
 'Développeur Angular H/F',
 'Développeurs FrontEnd - AngularJs | iOs | Android - DevOps',
 'ANALYSTE DEVELOPPEUR WEB (H/F)',
 'Staff Data Engineer',
 'Développeur PHP / Consu

In [8]:
df["Salary"].fillna("None", inplace = True)

In [9]:
df.shape

(4884, 9)

# Création Fonction Split_city

On sépare la ville et le cp. On remplace les differents arrondisements de Paris et Lyon par la ville. On remplis les valeurs manquantes par le départment de recherche

In [10]:
def split_city(row):
    location = row["Location"]
    if "Paris" in location or location == "France":
        city = "Paris"
        code_postal = "75"
    elif "Lyon" in location:
        city = "Lyon"
        code_postal = "69"
    elif "(" in location:
        split = location.split("(")
        city = split[0]
        code_postal = split[1]
    else:
        city = row["Department_Search"]
        code_postal="None"
    
    row["city"] = city
    row["cp"] = code_postal.replace(")","")
    return row

# Création colonne city et cp

In [11]:
df = df.apply(split_city, axis=1)

In [12]:
df[df["city"]=="None"].Department_Search.unique()

array([], dtype=object)

# Création Fonction Split Salary

Création des colonnes salaire_max, salaire_min et type de salaire. On transforme en annuelle seulement les valeurs en mois qui sont supérieur à 1500 smic brut

In [13]:
def split_salary(row):
    salary = row["Salary"]
    if "-" in salary:
        split = salary.split("-")
        salary_min = split[0]
        salary_max = split[1]
    else:
        salary_min = salary
        salary_max = salary
    
    row["salary_min"] = salary_min.replace("€","").replace("par heure","").replace("par mois","").replace("par semaine","").replace("par an","").replace(",50",".50").replace("\xa0","")
    row["salary_max"] = salary_max.replace("€","").replace("par heure","").replace("par jour","").replace("par mois","").replace("par semaine","").replace(",50",".50").replace("par an","").replace("\xa0","")
    
    
    if ("mois" in row["Salary"]) and (int(row["salary_min"]) > 1500):
        row["salary_min"] = int(row["salary_min"])*12
        row["salary_max"] = int(row["salary_max"])*12
        row["salary_type"] = "annually"
    
    if "mois" in row["Salary"] and (int(row["salary_min"]) <= 1500):
        row["salary_type"] = "monthly"
        
    if "jour" in row["Salary"]:
        row["salary_type"] = "daily"
    
    if "semaine" in row["Salary"]:
        row["salary_type"] = "weekly"
        
    if "an" in row["Salary"]:
        row["salary_type"] = "annually"
    
    if "heure" in row["Salary"]:
        row["salary_type"] = "per hour"
        
    return row

# Création des colonnes salary max, salary min et salary type

In [14]:
df_salary = df[df["Salary"] != "None"]
df_salary = df_salary.apply(split_salary, axis=1)
df_salary["salary_min"]=pd.to_numeric(df_salary["salary_min"],'coerce')
df_salary["salary_max"]=pd.to_numeric(df_salary["salary_max"],'coerce')
#df_salary["salary_mean"]=pd.to_numeric(df_salary["salary_mean"],'coerce')

In [15]:
df_salary

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,city,cp,salary_min,salary_max,salary_type
8,p_008d77a008590232,INGÉNIEUR DÉVELOPPEUR C++ / JAVA F/H,ALTEN,Toulouse (31),30 000 € - 45 000 € par an,Partenaire technologique de référence des plus...,08/09/2019,Développeur,Haute-Garonne,Toulouse,31,30000.0,45000.0,annually
15,p_00f2ad8db3c30b8e,Devops F/H,Kent FR,Bordeaux (33),35 000 € - 40 000 € par an,"KENT, marque de GroupAgora dédiée aux recrutem...",09/03/2019,Développeur,Gironde,Bordeaux,33,35000.0,40000.0,annually
17,p_00fa03a21ad52a71,Data Scientist H/F,QYSY,Paris (75),40 000 € - 65 000 € par an,QYSY est un cabinet de conseil et de recruteme...,29/09/2019,Data,75,Paris,75,40000.0,65000.0,annually
19,p_0130bef769eef3cc,BUSINESS DEVELOPER BTOB - DATA INTELLIGENCE (H/F),L.I.P,Lyon 3e (69),35 000 € - 60 000 € par an,69003 - Lyon-3e-Arrondissement CDI Bac+5 et pl...,07/10/2019,Data,Rhône,Lyon,69,35000.0,60000.0,annually
31,p_019d01b7494dc632,Référent fonctionnel et données - Data manager...,GARCIN FINANCES ET PARTICIPATIONS,Corbas (69),40 000 € - 45 000 € par an,Vous souhaitez participer à la structuration d...,10/09/2019,Data,Rhône,Corbas,69,40000.0,45000.0,annually
44,p_02074799b0dbc42e,Développeur PHP / Laravel,Sept Lieues,Paris (75),32 000 € - 37 000 € par an,PME de près de 20 ans d'existence dans la gest...,08/04/2019,Développeur,75,Paris,75,32000.0,37000.0,annually
49,p_02270c340e21e7ab,Développeur WinDev H/F,Axemploi Recrutement,Nantes (44),30 000 € - 40 000 € par an,DEVELOPPEUR WINDEV H/F Vous interviendrez sur...,18/09/2019,Développeur,Loire-Atlantique,Nantes,44,30000.0,40000.0,annually
66,p_033110c9c03332f9,Développeur Java H/F,WOM,Lyon (69),30 000 € - 40 000 € par an,"Notre client, spécialisé dans l’édition de log...",03/10/2019,Développeur,Rhône,Lyon,69,30000.0,40000.0,annually
87,p_046189897fdb70ed,CONTRÔLEUR DE GESTION - DATA ANALYST,SH Conseils,Paris 8e (75),35 000 € - 38 000 € par an,Pour le compte d’un Opérateur Télécoms de prem...,18/09/2019,Data,75,Paris,75,35000.0,38000.0,annually
105,p_05b4d08c39d1a13c,Développeur Mobile (Senior) iOS / Swift H/F,EXTERNATIC,Bordeaux (33),40 000 € - 50 000 € par an,"DESCRIPTION DE L'OFFRE Externatic, le hub d’op...",07/09/2019,Développeur,Gironde,Bordeaux,33,40000.0,50000.0,annually


In [16]:
print("Nombre de lignes avec salaire en années : {}".format(df_salary[df_salary["salary_type"]=="annually"].shape[0]))
print("Nombre de lignes avec salaire en mois : {}".format(df_salary[df_salary["salary_type"]=="monthly"].shape[0]))
print("Nombre de lignes avec salaire en semaine : {}".format(df_salary[df_salary["salary_type"]=="weekly"].shape[0]))
print("Nombre de lignes avec salaire en heure : {}".format(df_salary[df_salary["salary_type"]=="per hour"].shape[0]))

Nombre de lignes avec salaire en années : 675
Nombre de lignes avec salaire en mois : 30
Nombre de lignes avec salaire en semaine : 23
Nombre de lignes avec salaire en heure : 9


In [17]:
df_salary[df_salary["salary_type"]=="monthly"]

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,city,cp,salary_min,salary_max,salary_type
339,p_11932114a525d757,STAGE Chef de projet - Customer Data Managemen...,Danone,Limonest (69),1 120 € par mois,Danone recherche un(e) STAGE Chef de projet - ...,06/10/2019,Data,Rhône,Limonest,69,1120.0,1120.0,monthly
358,p_13477b94e2936148,Stage software et Data Engineer,Outmind,Paris 8e (75),600 € - 1 200 € par mois,À propos Outmind est une jeune startup créant ...,07/09/2019,Data,75,Paris,75,600.0,1200.0,monthly
599,p_21fa7fcf997add98,Stagiaire Stage Développeur/se web FRONT ou BACK,ITALIC,Paris 10e (75),700 € - 900 € par mois,"Récapitulatif du poste Le studio ITALIC , créé...",18/09/2019,Développeur,75,Paris,75,700.0,900.0,monthly
721,p_283edb6188ab0e56,Analyste développeur H/F,TMDS,Irigny (69),600 € - 1 000 € par mois,Stage QA Tester Pourquoi venir chez nous ? Tot...,04/10/2019,Développeur,Rhône,Irigny,69,600.0,1000.0,monthly
855,p_2f2cf27029d2954b,STAGIAIRE DEVELOPPEUR FRONT-END (H/F),LEOO,Paris (75),560 € par mois,"SOCIETE LEOO, filiale du groupe ADLPerformance...",07/09/2019,Développeur,75,Paris,75,560.0,560.0,monthly
895,p_31d61797124bb736,Data Analyst - Stage,Pretto,Paris (75),800 € - 1 200 € par mois,Dans une startup qui croît et se structure à t...,05/10/2019,Data,75,Paris,75,800.0,1200.0,monthly
1305,p_4adf45bdd9431ed7,"Data engineer (Oracle, SQL) / Freelance",Thiveo,Paris (75),600 € par mois,Mettre en uvre des solutions techniques sécur...,07/09/2019,Data,75,Paris,75,600.0,600.0,monthly
1377,p_4f36493edf4833b5,Stage - Développeur ASP.NET / C# (H/F),Aidimpact,Toulouse (31),580 € par mois,Développeur C# .NET Dans le cadre de notre cro...,27/09/2019,Développeur,Haute-Garonne,Toulouse,31,580.0,580.0,monthly
1402,p_50fd489da08f1f55,Développeur Web (stage) H/F,Blent.ai,Paris 3e (75),700 € par mois,Blent.ai est une startup spécialisée dans la...,07/10/2019,Développeur,75,Paris,75,700.0,700.0,monthly
1518,p_57f4a3ee1f4b1fd5,stage ingénieur - développeur,Certilience,Limonest (69),600 € - 850 € par mois,Résumé Environnement Vos missions Votre profil...,06/02/2019,Développeur,Rhône,Limonest,69,600.0,850.0,monthly


## Creation df par type de salaire

In [18]:
df_annually = df_salary[df_salary["salary_type"]=="annually"]
df_monthly = df_salary[df_salary["salary_type"]=="monthly"]
df_weekly = df_salary[df_salary["salary_type"]=="weekly"]
df_hour = df_salary[df_salary["salary_type"]=="per hour"]

In [19]:
df_monthly

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,city,cp,salary_min,salary_max,salary_type
339,p_11932114a525d757,STAGE Chef de projet - Customer Data Managemen...,Danone,Limonest (69),1 120 € par mois,Danone recherche un(e) STAGE Chef de projet - ...,06/10/2019,Data,Rhône,Limonest,69,1120.0,1120.0,monthly
358,p_13477b94e2936148,Stage software et Data Engineer,Outmind,Paris 8e (75),600 € - 1 200 € par mois,À propos Outmind est une jeune startup créant ...,07/09/2019,Data,75,Paris,75,600.0,1200.0,monthly
599,p_21fa7fcf997add98,Stagiaire Stage Développeur/se web FRONT ou BACK,ITALIC,Paris 10e (75),700 € - 900 € par mois,"Récapitulatif du poste Le studio ITALIC , créé...",18/09/2019,Développeur,75,Paris,75,700.0,900.0,monthly
721,p_283edb6188ab0e56,Analyste développeur H/F,TMDS,Irigny (69),600 € - 1 000 € par mois,Stage QA Tester Pourquoi venir chez nous ? Tot...,04/10/2019,Développeur,Rhône,Irigny,69,600.0,1000.0,monthly
855,p_2f2cf27029d2954b,STAGIAIRE DEVELOPPEUR FRONT-END (H/F),LEOO,Paris (75),560 € par mois,"SOCIETE LEOO, filiale du groupe ADLPerformance...",07/09/2019,Développeur,75,Paris,75,560.0,560.0,monthly
895,p_31d61797124bb736,Data Analyst - Stage,Pretto,Paris (75),800 € - 1 200 € par mois,Dans une startup qui croît et se structure à t...,05/10/2019,Data,75,Paris,75,800.0,1200.0,monthly
1305,p_4adf45bdd9431ed7,"Data engineer (Oracle, SQL) / Freelance",Thiveo,Paris (75),600 € par mois,Mettre en uvre des solutions techniques sécur...,07/09/2019,Data,75,Paris,75,600.0,600.0,monthly
1377,p_4f36493edf4833b5,Stage - Développeur ASP.NET / C# (H/F),Aidimpact,Toulouse (31),580 € par mois,Développeur C# .NET Dans le cadre de notre cro...,27/09/2019,Développeur,Haute-Garonne,Toulouse,31,580.0,580.0,monthly
1402,p_50fd489da08f1f55,Développeur Web (stage) H/F,Blent.ai,Paris 3e (75),700 € par mois,Blent.ai est une startup spécialisée dans la...,07/10/2019,Développeur,75,Paris,75,700.0,700.0,monthly
1518,p_57f4a3ee1f4b1fd5,stage ingénieur - développeur,Certilience,Limonest (69),600 € - 850 € par mois,Résumé Environnement Vos missions Votre profil...,06/02/2019,Développeur,Rhône,Limonest,69,600.0,850.0,monthly


## Création colonne salary mean pour chaque df

In [20]:
df_annually["salary_mean"] = (df_annually["salary_min"]+df_annually["salary_max"])/2
df_monthly["salary_mean"] = (df_monthly["salary_min"]+df_monthly["salary_max"])/2
df_weekly["salary_mean"] = (df_weekly["salary_min"]+df_weekly["salary_max"])/2
df_hour["salary_mean"] = (df_hour["salary_min"]+df_hour["salary_max"])/2

/Users/fabi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/fabi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/fabi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [21]:
df_hour

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,city,cp,salary_min,salary_max,salary_type,salary_mean
804,p_2c741ebd774063de,Concepteur Développeur Java Jee F/H,Orkeis,Nantes (44),30 € - 37 € par heure,"Intégré(e) au sein d’une équipe IT, vous inter...",08/12/2018,Développeur,Loire-Atlantique,Nantes,44,30.0,37.0,per hour,33.5
4358,pj_0af9a47690300a4f,Analyste d’exploitation informatique DevOps,NOVA TECHNOLOGY,Nantes (44),32 € - 35 € par heure,Nous recherchons actuellement un Analyste d’ex...,NaN,Développeur,Loire-Atlantique,Nantes,44,32.0,35.0,per hour,33.5
4369,pj_0fc43ce904d57fe3,Data Collector - Take Photos & Survey Parking ...,Inrix,Nantes (44),"13,50 € par heure",About Us : ParkMe licenses parking data for ca...,NaN,Data,Loire-Atlantique,Nantes,44,13.5,13.5,per hour,13.5
4409,pj_22353a43932262b7,architecte informatique Big Data,T&T Consulting,Paris (75),50 € par heure,Expériences et connaissances requises Dans un...,NaN,Data,75,Paris,75,50.0,50.0,per hour,50.0
4427,pj_2be08b65a54741c4,Data Collector - Take Photos & Survey Parking ...,Inrix,Lyon (69),"13,50 € par heure",About Us : ParkMe licenses parking data for ca...,NaN,Data,Rhône,Lyon,69,13.5,13.5,per hour,13.5
4452,pj_364c49cd87d41ad9,"Dev back-end Python / Start-up "" Data driven"" ...",SC Talent,Paris (75),50 € par heure,Celery / React / Django / Python Backend Devel...,NaN,Data,75,Paris,75,50.0,50.0,per hour,50.0
4580,pj_6fcad6df3b66f9f3,Ingénieur DevOps - H/F,SOLUTEC,Paris 8e (75),39 € par heure,Oracle / Capistrano / Ansible / Docker / Jenki...,NaN,Développeur,75,Paris,75,39.0,39.0,per hour,39.0
4868,pj_f7e3c209ffdd7369,JURISTE SENIOR EN DATA PRIVACY H/F,ADECCO,Villeurbanne (69),50 € par heure,The Adecco Group est le leader mondial des sol...,NaN,Data,Rhône,Villeurbanne,69,50.0,50.0,per hour,50.0
4875,pj_fc87c600387c3e85,Developpeur Java Back-end,Gentis Recruitment SAS,Paris (75),50 € par heure,"Bonjour, Notre client est une startup, jeune ...",NaN,Développeur,75,Paris,75,50.0,50.0,per hour,50.0


## Quantiles salary min annually

In [22]:
df_annually.salary_min.describe()

count      675.000000
mean     40998.761481
std      10930.715767
min      20000.000000
25%      33000.000000
50%      40000.000000
75%      47500.000000
max      90000.000000
Name: salary_min, dtype: float64

## Quantiles salary max annually

In [23]:
df_annually.salary_max.describe()

count       675.000000
mean      49769.296296
std       13259.715155
min       20388.000000
25%       40000.000000
50%       50000.000000
75%       55000.000000
max      120000.000000
Name: salary_max, dtype: float64

## Quantiles salary mean annually

In [24]:
annually_quantiles=df_annually.salary_mean.describe()
monthly_quantiles=df_monthly.salary_mean.describe() 
weekly_quantiles=df_weekly.salary_mean.describe()
hour_quantiles=df_hour.salary_mean.describe()

# Création colonne class label par rapport aux quantiles salary mean

In [25]:
def classification_annually(x):
    if x <= annually_quantiles["25%"]:
        label = 0
    elif x > annually_quantiles["25%"] and x <= annually_quantiles["50%"]:
        label = 1
    elif x > annually_quantiles["50%"] and x <= annually_quantiles["75%"]:
        label = 2
    else:
        label = 3
        
    return label

df_annually["salary_label"] = df_annually.salary_mean.apply(classification_annually)

/Users/fabi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [26]:
#def classification_min(x):
   # if x <= 30000:
        #label = 0
    #elif x > 30000 and x <= 40000:
        #label = 1
    #elif x > 40000 and x <= 47750:
        #label = 2
    #else:
        #label = 3
        
    #return label

#df_salary["label_salary_min"] = df_salary.salary_min.apply(classification_min)

In [27]:
#df_salary.drop(columns = 'label_salary_mean', inplace=True)

In [28]:
df_annually.sample(10)

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,city,cp,salary_min,salary_max,salary_type,salary_mean,salary_label
2159,p_7e9b60e398777877,DATA MANAGER SEQUENCAGE H/F,HOSPICES CIVILS DE LYON,Lyon (69),26 000 € par an,Le projet AURAGEN est un des deux projets rete...,07/09/2019,Data,Rhône,Lyon,69,26000.0,26000.0,annually,26000.0,0
4611,pj_817016fd795a26bd,Développeur Logiciel - H/F,MBP Digital,Nantes (44),28 000 € par an,Reactjs / Html / Php / Mysql / Jquery Frontend...,NaN,Développeur,Loire-Atlantique,Nantes,44,28000.0,28000.0,annually,28000.0,0
1291,p_49c0e878752ef08a,Développeur WinDev H/F,AXEMPLOI Recrutement,Nantes (44),30 000 € - 40 000 € par an,DEVELOPPEUR WINDEV H/F Vous interviendrez sur ...,17/09/2019,Développeur,Loire-Atlantique,Nantes,44,30000.0,40000.0,annually,35000.0,0
4709,pj_b0129ea913cb9659,ingénieur informatique Devops/Cloud (H/F),OVALO,Paris (75),45 000 € - 60 000 € par an,Le poste : Nous recherchons un Ingénieur Devop...,NaN,Développeur,75,Paris,75,45000.0,60000.0,annually,52500.0,3
195,p_09f8ad957b29d478,Développeur .Net H/F,effektiv,Lyon (69),38 000 € - 45 000 € par an,"Client Notre client, à mi-chemin entre la star...",07/09/2019,Développeur,Rhône,Lyon,69,38000.0,45000.0,annually,41500.0,1
2933,p_aebcdbafd8398f28,Technicien réseau Hotline Téléphonie Data (H/F),Manpower,Saint-Orens-de-Gameville (31),22 500 € - 24 000 € par an,L'entreprise Manpower TOULOUSE INFORMATIQUE re...,07/09/2019,Data,Haute-Garonne,Saint-Orens-de-Gameville,31,22500.0,24000.0,annually,23250.0,0
3530,p_d179e6455c960560,Développeurs mobile IOS & android indépendant ...,Sapiens,Nantes (44),32 000 € - 43 000 € par an,"NOTRE ENTREPRISE : Créé en 2003, SAPIENS est u...",17/09/2019,Développeur,Loire-Atlantique,Nantes,44,32000.0,43000.0,annually,37500.0,0
1583,p_5b8764b65acd1da8,Data Scientist H/F,CACIIS,Paris 15e (75),40 000 € - 55 000 € par an,"Depuis plus de 10 ans, CACIIS est un véritable...",07/09/2019,Data,75,Paris,75,40000.0,55000.0,annually,47500.0,2
2484,p_91f55b1a075d28e9,Data engineer AWS (H/F),Easy Partner,Paris (75),50 000 € - 55 000 € par an,### VOTRE PROFIL * Maîtrise de Scala * Esprit ...,07/09/2019,Data,75,Paris,75,50000.0,55000.0,annually,52500.0,3
1941,p_724cfbe357ad7b1f,Développement intégration SYSOPS/DEVOPS (F/H),MC2S,Paris 12e (75),40 000 € - 45 000 € par an,Poste concernant la prise en charge de l’inter...,08/11/2018,Développeur,75,Paris,75,40000.0,45000.0,annually,42500.0,1


# Création des df par tranches label

In [32]:
label_0 = df_salary[df_salary["salary_label"]==0]
label_1 = df_salary[df_salary["salary_label"]==1]
label_2 = df_salary[df_salary["salary_label"]==2]
label_3 = df_salary[df_salary["salary_label"]==3]

KeyError: 'salary_label'

In [68]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

### Création de la fonction qui retourne les mots relevants sur la colonne title

In [69]:
def clean_title(title):
    title = word_tokenize(title)
    #desc = [word.lower() for word in desc if word.isalpha() and len(word)>2]
    title = [word.lower() for word in title if (word.isalpha() or word.isdigit()) and len(word)>2]
    title = [word for word in title if word not in stop_words]
    return title

### On applique la fonction sur chaque df

In [70]:
stop_words = stopwords.words('french')
tag_title_0 = label_0["Title"].apply(clean_title)
tag_title_1 = label_1["Title"].apply(clean_title)    
tag_title_2 = label_2["Title"].apply(clean_title)
tag_title_3 = label_3["Title"].apply(clean_title)

NameError: name 'label_0' is not defined

In [71]:
from collections import Counter
result_0 = tag_title_0.apply(Counter).sum().items() # créer un dictionnaire avec la frequence de chaque mot de tag_title1
result_0 = sorted(result_0, key=lambda kv : kv[1], reverse=True) #on va ordonner par la valeur du dictionnaire en ordre descendant

result_1 = tag_title_1.apply(Counter).sum().items()
result_1 = sorted(result_1, key=lambda kv : kv[1], reverse=True)

result_2 = tag_title_2.apply(Counter).sum().items()
result_2 = sorted(result_2, key=lambda kv : kv[1], reverse=True)

result_3 = tag_title_3.apply(Counter).sum().items()
result_3 = sorted(result_3, key=lambda kv : kv[1], reverse=True)

NameError: name 'tag_title_0' is not defined

### Je crée le dictionnaire qui associe la frequence de chaque mot sur chaque df

In [ ]:
result_series_0 = {k: v for k,v in result_0}
result_series_1 = {k: v for k,v in result_1}
result_series_2 = {k: v for k,v in result_2}
result_series_3 = {k: v for k,v in result_3}

In [ ]:
result_series_0

# Visualisation du wordcloud title de chaque df

In [ ]:
from wordcloud import WordCloud

In [ ]:
wordcloud_0 = WordCloud().generate_from_frequencies(result_series_0)
wordcloud_1 = WordCloud().generate_from_frequencies(result_series_1)
wordcloud_2 = WordCloud().generate_from_frequencies(result_series_2)
wordcloud_3 = WordCloud().generate_from_frequencies(result_series_3)

In [ ]:
import matplotlib.pyplot as plt

figure = plt.figure(figsize = (25, 30))

plt.figure(1)
plt.subplot(4, 1, 1)
plt.imshow(wordcloud_0, interpolation="bilinear")
plt.axis("off")

plt.subplot(4, 1, 2)
plt.imshow(wordcloud_1, interpolation="bilinear")
plt.axis("off")

plt.subplot(4, 1, 3)
plt.imshow(wordcloud_2, interpolation="bilinear")
plt.axis("off")

plt.subplot(4, 1, 4)
plt.imshow(wordcloud_3, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
df_salary[df_salary['Title'].str.contains('Freelance')]

### Création de la fonction qui retourne les mots relevants sur la colonne desc

In [72]:
def clean_summary(desc):
    desc = word_tokenize(desc)
    #desc = [word.lower() for word in desc if word.isalpha() and len(word)>2]
    desc = [word.lower() for word in desc if (word.isalpha() or word.isdigit()) and len(word)>2]
    desc = [word for word in desc if (word not in stop_words_en) and (word not in stop_words_fr)]
    return desc

In [73]:
#stopwords
nltk.download('stopwords')
stop_words_en = stopwords.words('english')
stop_words_fr = stopwords.words('french')

[nltk_data] Downloading package stopwords to /Users/fabi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### on applique la fonction sur chaque df salary

In [74]:
tag_desc_0 = label_0["Description"].apply(clean_summary)
tag_desc_1 = label_1["Description"].apply(clean_summary)    
tag_desc_2 = label_2["Description"].apply(clean_summary)
tag_desc_3 = label_3["Description"].apply(clean_summary)

NameError: name 'label_0' is not defined

In [ ]:
tag_desc_3

# création des dictionnaires qui à chaque mot associe sa frequence

In [75]:
desc_0 = tag_desc_0.apply(Counter).sum().items()
desc_0 = sorted(desc_0, key=lambda kv : kv[1], reverse=True) 
dict_desc_0 = {k: v for k,v in desc_0}


desc_1 = tag_desc_1.apply(Counter).sum().items()
desc_1 = sorted(desc_1, key=lambda kv : kv[1], reverse=True) 
dict_desc_1 = {k: v for k,v in desc_1}

desc_2 = tag_desc_2.apply(Counter).sum().items()
desc_2 = sorted(desc_2, key=lambda kv : kv[1], reverse=True) 
dict_desc_2 = {k: v for k,v in desc_2}

desc_3 = tag_desc_3.apply(Counter).sum().items()
desc_3 = sorted(desc_3, key=lambda kv : kv[1], reverse=True) 
dict_desc_3 = {k: v for k,v in desc_3}

NameError: name 'tag_desc_0' is not defined

# Visualisation du wordcloud job_desc pour chaque df

In [ ]:
wordcloud5 = WordCloud().generate_from_frequencies(dict_desc_0)
wordcloud6 = WordCloud().generate_from_frequencies(dict_desc_1)
wordcloud7 = WordCloud().generate_from_frequencies(dict_desc_2)
wordcloud8 = WordCloud().generate_from_frequencies(dict_desc_3)

figure = plt.figure(figsize = (25, 30))
plt.figure(1)
plt.subplot(4, 1, 1)
plt.imshow(wordcloud5, interpolation="bilinear")
plt.axis("off")

plt.subplot(4, 1, 2)
plt.imshow(wordcloud6, interpolation="bilinear")
plt.axis("off")

plt.subplot(4, 1, 3)
plt.imshow(wordcloud7, interpolation="bilinear")
plt.axis("off")

plt.subplot(4, 1, 4)
plt.imshow(wordcloud8, interpolation="bilinear")
plt.axis("off")
plt.show()

## prediction salaire avec job desc brut seulement

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [76]:
X_desc = df_annually.Description.str.replace("\n"," ").str.replace(r'\b\d\d\W?\d\d\d\b','').str.replace(r'\+',' plus ')
y = df_annually.salary_label

In [77]:
stop_words_en_fr = stop_words_en + stop_words_fr + ["h","f", "mois", "ci", "e"]

In [78]:
cv = CountVectorizer(token_pattern = r'\b\w+\b',stop_words=stop_words_en_fr,  ngram_range=(1, 4), min_df = 2)
cv.fit(X_desc)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 4), preprocessor=None,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',...', 'aient', 'eusse', 'eusses', 'eût', 'eussions', 'eussiez', 'eussent', 'h', 'f', 'mois', 'ci', 'e'],
        strip_accents=None, token_pattern='\\b\\w+\\b', tokenizer=None,
        vocabulary=None)

In [41]:
len(cv.get_feature_names())

57070

In [42]:
X_desc_trans = cv.transform(X_desc)
#X_desc_trans = pd.DataFrame(cv.transform(X_desc).todense(), columns=cv.get_feature_names())
#X_desc_trans

In [43]:
import numpy as np
#np.asmatrix(X_desc_trans)

In [44]:
#word_counts = X_desc_trans.sum(axis=0)
#word_counts.sort_values(ascending = False).head(20)

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(X_desc_trans, y, test_size=0.3, random_state=0, stratify=y)


rfc = RandomForestClassifier(300, random_state=0)
rfc.fit(X_train, y_train)

rfc.score(X_test,y_test)

0.5714285714285714

In [46]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = cv.get_feature_names()).reset_index()
feature_importances.columns = ['feature', 'importance']

feature_importances.sort_values('importance', ascending=False).head(40)

,feature,importance
18595,développeur,0.003421
14443,data,0.002670
42088,python,0.002490
26006,informatique,0.002305
6710,aws,0.002216
35455,paris,0.002095
15349,devops,0.002076
47432,similaire 1,0.001864
56232,équipes,0.001566
53236,type,0.001512


In [47]:
#list(X_desc[X_desc.str.contains("10 ans")])

# Prédiction salaire avec Title seulement

In [79]:
X_title = df_annually.Title.str.replace("\n"," ").str.replace("/"," ")

In [49]:
#except number and alpha numerical caracter, ngram_range --> ici : regroupement de 1à2 mots
cv2 = CountVectorizer( token_pattern=r'\b[^\d\W]+\b', stop_words=stop_words_en_fr, ngram_range=(1, 2))
cv2.fit(X_title)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',...', 'aient', 'eusse', 'eusses', 'eût', 'eussions', 'eussiez', 'eussent', 'h', 'f', 'mois', 'ci', 'e'],
        strip_accents=None, token_pattern='\\b[^\\d\\W]+\\b',
        tokenizer=None, vocabulary=None)

In [80]:
print(stop_words)

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

In [81]:
print(stop_words_en_fr)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [50]:
cv2.get_feature_names()

['accepté',
 'access',
 'access cdi',
 'acteur',
 'acteur leader',
 'adelia',
 'administrateur',
 'administrateur système',
 'administrateur systèmes',
 'administrateur sytèmes',
 'administration',
 'administration système',
 'adminsys',
 'adminsys linux',
 'affaires',
 'affaires expert',
 'affaires ia',
 'agence',
 'agence web',
 'agile',
 'agile nantes',
 'agile paris',
 'agile vue',
 'ambassadeur',
 'ambassadeur formateur',
 'analyst',
 'analyst administration',
 'analyst analyste',
 'analyst bi',
 'analyst cdi',
 'analyst chef',
 'analyst confirmé',
 'analyst crm',
 'analyst entrepôt',
 'analyst foodtech',
 'analyst junior',
 'analyst marketplace',
 'analyst mobile',
 'analyste',
 'analyste bi',
 'analyste cobol',
 'analyste developpeur',
 'analyste développeur',
 'analyste informatique',
 'analyste éditeur',
 'analytics',
 'analytics manager',
 'analytique',
 'android',
 'android indépendant',
 'android ios',
 'android web',
 'angular',
 'angular confirme',
 'angular editeur',
 'a

In [51]:
X_title_trans = cv2.transform(X_title)
#X_title_trans = pd.DataFrame(cv2.transform(X_title).todense(), columns=cv2.get_feature_names())
#X_title_trans

In [52]:
#word_counts2 = X_title_trans.sum(axis=0)
#word_counts2.sort_values(ascending = False).head(30)

In [53]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_title_trans, y, test_size=0.3, random_state=10, stratify=y)


rfc = RandomForestClassifier(50, random_state=0)
rfc.fit(X_train2, y_train2)

rfc.score(X_test2,y_test2)

0.4876847290640394

In [54]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = cv2.get_feature_names()).reset_index()
feature_importances.columns = ['feature', 'importance']

feature_importances.sort_values('importance', ascending=False).head(20)

,feature,importance
358,développeur,0.023966
207,data,0.016159
755,lead,0.014799
286,devops,0.011285
1059,senior,0.011002
471,engineer,0.010785
657,ingénieur,0.009973
622,informatique,0.009874
698,java,0.008786
798,manager,0.008729


# Prédiction salaire avec ville

In [55]:
df_annually.city.unique()

array(['Toulouse ', 'Bordeaux ', 'Paris', 'Lyon', 'Corbas ', 'Nantes ',
       'Pessac ', 'Limonest ', 'Portet-sur-Garonne ', 'Blagnac ',
       'Talence ', 'Villeneuve-lès-Bouloc ', 'Mérignac ', 'Marennes ',
       'Rillieux-la-Pape ', 'Canéjan ', 'Beauzelle ',
       "Saint-Didier-au-Mont-d'Or ", 'Villeurbanne ', 'Lissieu ',
       'Gradignan ', 'Treillières ', 'Loire-Atlantique', 'Limas ',
       'Rhône', 'Labège ', 'Saint-Pierre-de-Chandieu ', 'Saint-Nazaire ',
       'Ancenis ', 'Vénissieux ', 'Chassieu ', 'Dardilly ',
       'Saint-Orens-de-Gameville ', 'Bruges ', 'Brignais ',
       'Saint-Herblain ', 'Châteaubriant ', 'Francheville ', 'Bron ',
       'Carquefou ', 'Toussieu ', 'Charbonnières-les-Bains ',
       'Saint-Médard-en-Jalles ', 'Le Haillan ', 'Castanet-Tolosan ',
       "L'Union ", 'Tassin-la-Demi-Lune ', 'Écully '], dtype=object)

In [56]:
city_d = pd.get_dummies(df_annually.city)
X_city = city_d

X_train, X_test, y_train, y_test = train_test_split(X_city, y, test_size=0.3, random_state=0, stratify=y)

In [57]:
rfc = RandomForestClassifier(300, random_state=0)
rfc.fit(X_train, y_train)

rfc.score(X_test,y_test)

0.46798029556650245

In [58]:
X_city.columns

Index(['Ancenis ', 'Beauzelle ', 'Blagnac ', 'Bordeaux ', 'Brignais ', 'Bron ',
       'Bruges ', 'Canéjan ', 'Carquefou ', 'Castanet-Tolosan ',
       'Charbonnières-les-Bains ', 'Chassieu ', 'Châteaubriant ', 'Corbas ',
       'Dardilly ', 'Francheville ', 'Gradignan ', 'L'Union ', 'Labège ',
       'Le Haillan ', 'Limas ', 'Limonest ', 'Lissieu ', 'Loire-Atlantique',
       'Lyon', 'Marennes ', 'Mérignac ', 'Nantes ', 'Paris', 'Pessac ',
       'Portet-sur-Garonne ', 'Rhône', 'Rillieux-la-Pape ',
       'Saint-Didier-au-Mont-d'Or ', 'Saint-Herblain ',
       'Saint-Médard-en-Jalles ', 'Saint-Nazaire ',
       'Saint-Orens-de-Gameville ', 'Saint-Pierre-de-Chandieu ', 'Talence ',
       'Tassin-la-Demi-Lune ', 'Toulouse ', 'Toussieu ', 'Treillières ',
       'Villeneuve-lès-Bouloc ', 'Villeurbanne ', 'Vénissieux ', 'Écully '],
      dtype='object')

In [59]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_city.columns).reset_index()
feature_importances.columns = ['feature', 'importance']

feature_importances.sort_values('importance', ascending=False)

,feature,importance
28,Paris,0.391325
3,Bordeaux,0.066871
24,Lyon,0.061962
27,Nantes,0.060437
41,Toulouse,0.056672
23,Loire-Atlantique,0.030471
40,Tassin-la-Demi-Lune,0.024076
34,Saint-Herblain,0.022183
31,Rhône,0.019068
18,Labège,0.016786


In [60]:
from sklearn.model_selection import GridSearchCV
#n_estimators = np.linspace(start = 600 , stop = 1000, num = 8, dtype= int )
#n_estimators
n_estimators = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

param_grid = {'n_estimators': n_estimators}
grid = GridSearchCV(rfc, param_grid =param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.0f" % (grid.best_params_, grid.best_score_ * 100 ))
print( "Best estimator accuracy on test set {:.2f} ".format(grid.best_estimator_.score(X_test, y_test) * 100 ) )

The best parameters are {'n_estimators': 50} with a score of 46
Best estimator accuracy on test set 46.80 


In [61]:
df_annually.Department_Search.unique()

array(['Haute-Garonne', 'Gironde', '75', 'Rhône', 'Loire-Atlantique'],
      dtype=object)

In [62]:
depart_d = pd.get_dummies(df_annually.Department_Search)
X_depart = depart_d

X_train, X_test, y_train, y_test = train_test_split(X_depart, y, test_size=0.3, random_state=0, stratify=y)

n_estimators = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

param_grid = {'n_estimators': n_estimators}
grid = GridSearchCV(rfc, param_grid =param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.0f" % (grid.best_params_, grid.best_score_ * 100 ))
print( "Best estimator accuracy on test set {:.2f} ".format(grid.best_estimator_.score(X_test, y_test) * 100 ) )

The best parameters are {'n_estimators': 50} with a score of 45
Best estimator accuracy on test set 44.83 


In [63]:
rfc = RandomForestClassifier(50, random_state=0)
rfc.fit(X_train, y_train)

feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X_depart.columns).reset_index()
feature_importances.columns = ['feature', 'importance']

feature_importances.sort_values('importance', ascending=False)

,feature,importance
0,75,0.551818
4,Rhône,0.172739
2,Haute-Garonne,0.120490
3,Loire-Atlantique,0.104565
1,Gironde,0.050389


In [64]:
df_annually.to_csv("df_annually.csv")

# Prediction avec les X_desc_trans, X_title_trans et X_city

In [107]:
df_annually.head(1)

,_id,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,city,cp,salary_min,salary_max,salary_type,salary_mean,salary_label
8,p_008d77a008590232,INGÉNIEUR DÉVELOPPEUR C++ / JAVA F/H,ALTEN,Toulouse (31),30 000 € - 45 000 € par an,Partenaire technologique de référence des plus...,08/09/2019,Développeur,Haute-Garonne,Toulouse,31,30000.0,45000.0,annually,37500.0,0


In [108]:
#Pandas describe() is used to view some basic statistical details like percentile, mean, std etc. of a data frame or a series of numeric values. 
df_annually.describe()

,salary_min,salary_max,salary_mean,salary_label
count,675.000000,675.000000,675.000000,675.000000
mean,40998.761481,49769.296296,45384.028889,1.388148
std,10930.715767,13259.715155,11569.392983,1.140244
min,20000.000000,20388.000000,20388.000000,0.000000
25%,33000.000000,40000.000000,37500.000000,0.000000
50%,40000.000000,50000.000000,45000.000000,1.000000
75%,47500.000000,55000.000000,51250.000000,2.500000
max,90000.000000,120000.000000,100000.000000,3.000000


In [110]:
#number of line
df_annually.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 675 entries, 8 to 4879
Data columns (total 16 columns):
_id                  675 non-null object
Title                675 non-null object
Company              675 non-null object
Location             675 non-null object
Salary               675 non-null object
Description          675 non-null object
Date                 546 non-null object
Job_Search           675 non-null object
Department_Search    675 non-null object
city                 675 non-null object
cp                   675 non-null object
salary_min           675 non-null float64
salary_max           675 non-null float64
salary_type          675 non-null object
salary_mean          675 non-null float64
salary_label         675 non-null int64
dtypes: float64(3), int64(1), object(12)
memory usage: 89.6+ KB


In [100]:
df_annually.groupby('Description')['_id'].nunique().count()

669

In [103]:
df_annually.duplicated(subset=None, keep='first').count()

675

In [104]:
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = df_annually[df_annually.duplicated()]
 
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [_id, Title, Company, Location, Salary, Description, Date, Job_Search, Department_Search, city, cp, salary_min, salary_max, salary_type, salary_mean, salary_label]
Index: []


In [94]:
#df_annually._groupby.count()
df_annually.groupby(['_id']).count() 

,Title,Company,Location,Salary,Description,Date,Job_Search,Department_Search,city,cp,salary_min,salary_max,salary_type,salary_mean,salary_label
_id,,,,,,,,,,,,,,,
p_008d77a008590232,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
p_00f2ad8db3c30b8e,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
p_00fa03a21ad52a71,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
p_0130bef769eef3cc,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
p_019d01b7494dc632,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
p_02074799b0dbc42e,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
p_02270c340e21e7ab,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
p_033110c9c03332f9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
p_046189897fdb70ed,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [65]:
X = df.iloc[:,[3,-1]].values

# clustering dataset
# determine k using elbow method
# k means determine K
# K = nombre de clusters à déterminer
inertias = []
K = range(2,10) 

for k in K:
    kmeanModel = KMeans(init = 'k-means++', n_clusters=k)
    kmeanModel.fit(X) #calcule l'inertie intra-classe, labellise
    inertias.append(kmeanModel.inertia_) #on récupère que l'inertie

# Plot the elbow
plt.plot(K, inertias, 'bx-') #'bx-' : style de ligne
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

NameError: name 'KMeans' is not defined